In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling
from visualize import visualize_rooms
from preprocessing import preprocess_df, merge_data
from sklearn.ensemble import RandomForestRegressor
from model import train_model
from autots import AutoTS

In [ ]:
from autots.evaluator.benchmark import Benchmark
bench = Benchmark()
bench.run(n_jobs="auto", times=3)
bench.results

In [ ]:
# load dataset and preprocess
df = pd.read_csv("data/train/data.csv", sep=";")
# merge dataset with weather data
df = merge_data(df)

In [ ]:
# discretize, impute etc.
dfp = preprocess_df(df.copy())
print(dfp.isna().sum())


In [ ]:
# export
import torch
dfp.to_csv("./data/train/preprocessed_train.csv")
print(torch.cuda.is_available())

In [2]:
# import
df = pd.read_csv("./data/train/preprocessed_train.csv")

In [3]:
df = df.rename(columns={'Unnamed: 0': 'timestamp'})
#df.set_index('timestamp', inplace=True)

In [4]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss, RMSE
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [5]:
g = df.groupby("Room")
split = [g.get_group(x) for x in g.groups]
dfone = split[0].reset_index(drop=True)
dfone["time_idx"] = dfone.index

In [ ]:
print(dfone.columns)

In [ ]:
print(dfone["RoomTemperature"])

In [6]:
max_prediction_length = 4*24*7
max_encoder_length = 5*4*24*7
training_cutoff = dfone["time_idx"].max() - max_prediction_length

training = TimeSeriesDataSet(
    dfone[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="RoomTemperature",
    group_ids=["Room"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=[],
    #static_reals=["Room"],
    time_varying_known_categoricals=[],
    time_varying_unknown_reals=["RoomTemperature"],
    #time_varying_known_reals=[
    #"time_idx",  'AirqualityPerc', 'HeaterPerc',
    #    'CoolerPerc', 'TempSupplyAir', 'RelativeHumiditySupplyAir',
    #    'HeatingPower', 'CoolingPower', 'AirTemperature', 'WindDirection',
    #    'BrightnessNorth', 'BrightnessEast', 'BrightnessSouth',
    #    'BrightnessWest', 'dwpt', 'rhum',
    #    'prcp', 'wspd', 'tsun'],
    # time_varying_unknown_categoricals=[],
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

validation = TimeSeriesDataSet.from_dataset(training, dfone, predict=True, stop_randomization=True)
batch_size = 32  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=10)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=10)

In [ ]:
actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
(actuals - baseline_predictions).abs().mean().item()

In [7]:
# configure network and trainer
pl.seed_everything(42)
trainer = pl.Trainer(
    gpus=0,
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    hidden_size=16,
    lstm_layers=2,
    learning_rate=0.06760829753919811,
    dropout=0.1,
    loss=RMSE(),
    attention_head_size=4
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

Global seed set to 42
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Number of parameters in network: 21.5k


In [ ]:
res = trainer.tuner.lr_find(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    max_lr=10.0,
    min_lr=1e-6,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

In [8]:
# configure network and trainer
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

trainer = pl.Trainer(
    max_epochs=50,
    gpus=0,
    weights_summary="top",
    gradient_clip_val=0.1,
    limit_train_batches=30,  # coment in for training, running valiation every 30 batches
    #fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)


   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | RMSE                            | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 0     
3  | prescalers                         | ModuleDict                      | 80    
4  | static_variable_selection          | VariableSelectionNetwork        | 1.7 K 
5  | encoder_variable_selection         | VariableSelectionNetwork        | 1.2 K 
6  | decoder_variable_selection         | VariableSelectionNetwork        | 528   
7  | static_context_variable_selection  | GatedResidualNetwork            | 1.1 K 
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 1.1 K 
9  | static_context_initial_cell_lstm   | GatedResidualNetwork            | 1.1 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [ ]:
df_ts = df.copy()
df_ts = df_ts.reset_index().rename(columns={'index':'timestamp'})
print(df_ts.head(2))
# Evaluate different simple time series models
model = AutoTS(
    forecast_length=3,
    frequency='infer',
    ensemble='simple',
    max_generations=3,
    num_validations=2,
    n_jobs='auto'
)

model.fit(df_ts, date_col='timestamp', value_col='RoomTemperature', id_col="Room")